# Prepare notebook

In [1]:
%load_ext dotenv
%load_ext autoreload
%autoreload 2
import os
# move current working directory up two levels to root
# not pretty but this is a notebook
# don't run this cell more than once or you'll move another two directories up, which wouldn't be good
os.chdir(os.pardir); os.chdir(os.pardir); os.chdir(os.pardir)
print('Current working directory is %s' % os.getcwd())
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from multivac.src import utilities
from multivac import settings
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from collections import OrderedDict
import copy
import re
import feedparser
import pubmed_parser
import time
import slate
import re
import pickle
import json

Current working directory is E:\Users\Peter_Rasmussen\gh
Directory multivac\data already exists
Directory multivac\data\raw already exists
Directory multivac\data\interim already exists
Directory multivac\data\processed already exists
Directory multivac\data\processed\metadata already exists
Directory multivac\models already exists
Directory multivac\data\raw\arxiv already exists
Directory multivac\data\raw\pubmed already exists
Directory multivac\data\raw\springer already exists


In [2]:
def parse_pdf(src):
    """Parse """
    try:
        # try to open file
        with open(src, 'rb') as f:
            doc = slate.PDF(f)

        # get text: strip out newlines and extra spaces
        doc = ' '.join([' '.join(x.split()) for x in doc])
        text = doc.split(' Abstract ')[-1].split(' Acknowledgments ')[0].split(' ∗ ∗ ∗ ')[0].strip()

    except:  #  PDFSyntaxError
        text = None

    return text


def parse_html(src):

    with open(src, 'r', encoding='utf-8') as f:
        raw_data_ = f.read()
    soup = bs(raw_data_)
    try:
        text = ' '.join(soup.find('article').get_text().split())
    except AttributeError:
        text = None
    return text


def parse_articles_data(source, data_raw_dir, verbose=False):
    """Parse Arxiv and Springer article data."""
    # load metadata
    fn = source + '.pkl'
    metadata_src = settings.metadata_dir / fn
    with open(metadata_src, 'rb') as f:
        metadata_ = pickle.load(f)

    # we'll just add the text to a new arxiv object, an ordered dict keyed on doi or other id
    data = OrderedDict()
    srcs = [data_raw_dir / x for x in os.listdir(data_raw_dir)]
    for ix, article_metadata in enumerate(metadata_):
        
        # initialize temp dictionary
        temp = OrderedDict()
        temp['metadata'] = copy.deepcopy(article_metadata)
        temp['metadata']['source'] = source
        article_fn = article_metadata['fn']
        if verbose:
            print(article_fn)
        src = data_raw_dir / article_fn
    
        # define key and value
        if source == 'arxiv':
            k = article_metadata['fn'].strip('.pdf')
            temp['text'] = parse_pdf(src)
        elif source =='springer':
            k = article_metadata['doi']
            temp['text'] = parse_html(src)
        elif source == 'pubmed':
            raise ValueError('pubmed not supported. Only "arxiv" and "springer" supported. Try "parse_pubmed() function"')
        else:
            raise ValueError('Only "arxiv" and "springer" supported as sources.')
        
        # populate interim dictionary
#         if len()
        data[k] = temp
    
    # save intermediate outputs
    data_interim_dst = settings.interim_dir / fn
    with open(data_interim_dst, 'wb') as f:
        pickle.dump(data, f)
    return data


def parse_pubmed(src):
    """Parse pubmed xml article data and return metadata and text."""
    metadata = pubmed_parser.parse_pubmed_xml(src)
    text = pubmed_parser.parse_pubmed_paragraph(src, all_paragraph=True)
    text = ' '.join(' '.join([x['text'] for x in text]).split())
    return metadata, text


def aggregate_pubmed(srcs, verbose=False):
    """Aggregate a set of Pubmed article text and metadata."""
    pubmed_data = OrderedDict()
    for src in srcs:
        if verbose:
            print(src)
        try:
            temp = OrderedDict()
            metadata, text = parse_pubmed(str(src.absolute()))
            temp['metadata'] = metadata
            temp['metadata']['source'] = 'pubmed'
            temp['text'] = text
            try:
                k = metadata['doi']
            except AttributeError:
                k = src.strip('.xml')
            if len(text) > 0:
                pubmed_data[k] = temp
            print(src)
        except:
            if verbose:
                print('Error: %s' % src)
            pass
    return pubmed_data

In [ ]:
output = {}
for source in settings.sources:
    print(source)
    fn = source + '.pkl'
    data_raw_dir = settings.raw_dir / source
    if source in ['arxiv', 'springer']:
        data = parse_articles_data(source, data_raw_dir)
    elif source == 'pubmed':
        srcs = [data_raw_dir / x for x in os.listdir(data_raw_dir)]
        data = aggregate_pubmed(srcs)
    if len(output) == 0:
        output = copy.deepcopy(data)
    else:
        output.update(data)

In [ ]:
# combine each source into one
output = copy.deepcopy(pubmed_data)
output.update(arxiv_data)
output.update(springer_data)

In [ ]:
# save outputs
dst_dir = os.path.join(r'L:\Atlanta-30_days_only', 'Domonique'); utilities.mkdir(dst_dir)
dst = os.path.join(dst_dir, 'data.json')
with open(dst, 'w') as f:
    json.dump(output, f)

In [ ]:
print(len(output))
output.keys()


In [ ]:
len(pubmed_data)

In [ ]:
# # load metadata
# with open(settings.metadata_dir / 'arxiv.pkl', 'rb') as f:
#     arxiv_metadata_ = pickle.load(f)

# we'll just add the text to a new arxiv object, an ordered dict keyed on doi or other id
# arxiv_data = OrderedDict()

# arxiv_srcs = [settings.arxiv_dir / x for x in os.listdir(settings.arxiv_dir)]

# for ix, article_metadata in enumerate(arxiv_metadata_[109:]):
#     fn = article_metadata['fn']
#     # define key, usually the doi
#     k = article_metadata['id'].split('/abs/')[-1]
    
#     # initialize temp dictionary
#     temp = OrderedDict()
#     temp['metadata'] = copy.deepcopy(article_metadata)
#     src = settings.arxiv_dir / fn
#     temp['data'] = parse_pdf(src)
    
#     arxiv_data[k] = temp
    
dst = settings.interim_dir / 'arxiv.pkl'
with open(dst, 'wb') as f:
    pickle.dump(arxiv_data, f)

# Springer

In [ ]:
srcs = [settings.springer_dir / x for x in os.listdir(settings.springer_dir)]

for src in srcs[:1]:


In [ ]:
data

In [ ]:
with open(settings.metadata_dir / 'springer.pkl', 'rb') as f:
    metadata_ = pickle.load(f)

In [ ]:
metadata_[0]['fn']

In [ ]:
# 
for ix, md in enumerate(springer_metadata):
    fn = md['doi'].replace('/', '-')
    if len(fn) == 0:
        fn = md['identifier']
    r = requests.get(md['url'][0]['value'])
    html = bs(r.text).encode('utf-8').decode('utf-8')
    dst = settings.raw_dir / 'springer' / (fn + '.html')
    with open(dst, 'w', encoding='utf-8') as f:
        f.write(html)
    time.sleep(0.3)

In [ ]:
soup = bs(r.text)

In [ ]:
li = [x.contents[0] for x in soup.find('article').find_all('p', {'class':'Para'})]
li

In [ ]:
# print(soup.find('article').prettify())
li = [x.get_text() for x in soup.find('article').find_all('p')]
s = 'Springer Nature remains neutral '
li

In [ ]:
text = pubmed_parser.parse_pubmed_paragraph(str((settings.raw_dir / 'springer' / 'test.xml').absolute()), all_paragraph=True)

In [ ]:
r = requests.get(springer_metadata[1]['url'][0]['value'])


# IEEE Xplore

In [ ]:
base = 'http://ieeexploreapi.ieee.org/api/v1/search/articles?'
url = base + q
params = {'max_records': 20, 'start_record': 1, 'querytext': q, 'apikey': ieee_api_key}
params_ = copy.deepcopy(params)

ieee_metadata = []
while True:
    r = requests.get(url, params_)
    if params_['start_record'] > r.json()['total_records']:
        break
    for article in r.json()['articles']:
        if i['access_type'] != 'LOCKED':
            ieee_metadata.append(article)
    params_['start_record'] = params_['start_record'] + params_['max_records']
    time.sleep(wait_time)
print(len(ieee_metadata))

# Pubmed (Entrez)
* https://marcobonzanini.com/2015/01/12/searching-pubmed-with-python/

In [ ]:
# Download all of it
# get the ids
url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc&term=%22sir+model%22&set=pmc-open&metadataPrefix=pmc&usehistory=y&retmax=2000'


In [ ]:
# search pubmed central for free full text articles containing selected query
terms = ['sir model', 'susceptible-infected-recovered']
replace = lambda s: s.replace(' ', '+')
quote = lambda s: '%22' + s + '%22'
terms = [quote(replace(s)) for s in terms]
term = 'term='+ '%28'+ '+OR+'.join(terms) + '%29'
fulltext = 'free+fulltext%5bfilter%5d'
retmax = 'retmax=2000'
base = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc'
params = {'retmax': 2000, 'email': }
url = base + '&' + term + '+' + fulltext + '&' + retmax
r = requests.get(url)
ids = [x.contents[0] for x in bs(r.text).find_all('id')]
ids[5]

In [ ]:
len(ids)

In [ ]:
pmc_articles = OrderedDict()
for i in ids:
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=' + str(i)
    r = requests.get(url, params={'id': i})
    xml = r.text
    pmc_id = 'pmc' + str(i)
    fn = (pmc_id + '.xml')
    path = settings.raw_dir / 'pubmed' / fn
    with open(path, 'w') as f:
        f.write(xml)
    time.sleep(0.5)

In [ ]:
if len(text) > 0:
    pmc_articles[doi] = {'metadata': metadata, 'text': text}

text = pubmed_parser.parse_pubmed_paragraph(str(path.absolute()), all_paragraph=True)
metadata = pubmed_parser.parse_pubmed_xml(str(path.absolute()))
doi = metadata.pop('doi')


In [ ]:
pmc_articles[doi]

In [ ]:
len(pmc_articles)

In [ ]:
pmc_articles

In [ ]:
paragraphs

In [ ]:
# r = requests.get(url, params={'id': i})


In [ ]:
src = r'E:\Users\Peter_Rasmussen\gh\multivac\data\raw\pubmed\pmc4760143'

In [ ]:
# the parsing answer: https://github.com/titipata/pubmed_parser


In [ ]:
url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=6214536'
r = requests.get(url)

In [ ]:
pubmed_parser.parse_xml_web(ids[28], save_xml=False)

In [ ]:
xml = r.text

In [ ]:
imp

In [ ]:
import pubmed_parser

In [ ]:
dicts_out = pp.parse_pubmed_paragraph('data/6605965a.nxml', all_paragraph=False)

In [ ]:
print(xml.keys())

In [ ]:
oa_file_list_ = pd.read_csv('ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_file_list.csv')

In [ ]:
oa_file_list_['Article Citation'].unique()

In [ ]:
# tbd

url = 'https://www.ncbi.nlm.nih.gov/pmc/oai/oai.cgi?verb=ListRecords&metadataPrefix=pmc&term=%22sir+model%22+OR%22susceptible+-+infected+-+recovered%22&set=pmc-open'
r = requests.get(url)

In [ ]:
r.text[:1000]

# Old

In [ ]:



# def process_article_metadata(metadata):
#     """Extract metadata for one article and organize metadata into a dictionary.
#     Inputs:
#         metadata    String; Read in from API call
#     Outputs:
#         d  Dictionary of article metadata
#     """

#     # for each article extract and organize metadata
#     metadata = bs(metadata.strip(), 'lxml')

#     # each article's metadata contained in a dictionary
#     d = {}

#     # add single-value attributes
#     single_value_attributes = ['id', 'updated', 'published', 'title', 'summary', 'doi']
#     for single_value_attribute in single_value_attributes:
#         try:
#             v = metadata.find(single_value_attribute).contents[0].strip()
#         except AttributeError:
#             # some articles don't have doi numbers so fall back on arxiv doi
#             if single_value_attribute=='doi':
#                 v = d['id']
#             else:
#                 v = None
#         d[single_value_attribute] = v

#     # add multiple-value attributes and edge-case single-value attributes
#     d['arxiv:primary_category'] = metadata.find('arxiv:primary_category').attrs['term']
#     d['arxiv_categories'] = [x['term'] for x in metadata.find_all('category')]
#     d_links = {}
#     for link in metadata.find_all('link'):
#         if 'title' in link.attrs:
#             k, v = link.attrs['title'], link.attrs['href']
#             d_links[k] = v
#     d['links'] = d_links
#     d['authors'] = [x.contents[0] for x in metadata.find_all('name')]

#     return d


# def get_metadata_from_page(xml_text):
#     """
#     Usage of output:
#         As an OrderedDict, d_page_metadata has the usual dictionary functionality
#         It can also be accessed like a list using the approach below (Python 3 approach below):
#             items = list(d_page_metadata.items())
#             items[0]
#     """
    
#     # prep metadata returned by api query
#     articles_metadata = re.sub(' +', ' ', xml_text.replace('arxiv:doi', 'doi').replace('\n', ' ')).strip().split('<entry>')[1:]

#     # iterate over each article and extract and organize metadata
#     d_page_metadata = OrderedDict()
#     for article_metadata in articles_metadata:
#         v = copy.deepcopy(process_article_metadata(article_metadata))
#         k = v.pop('doi')
#         d_page_metadata[k] = v
    
#     return d_page_metadata



In [ ]:
# ps2ascii, gzip
import gzip
import os

In [ ]:
f = gzip.open(raw_src, 'rb')

In [ ]:
ps = f.read()

In [ ]:
f.close()

In [ ]:
raw_dir = r'E:\Users\Peter_Rasmussen\gh\multivac\data\raw\arxiv'
raw_src = os.path.join(raw_dir, '1411.2370v2.ps.gz')
# with gzip.open(raw_src, 'rb') as f:
#     ps = f.read()
    
def opener(filename):
    f = open(filename,'rb')
    if (f.read(2) == '\x1f\x8b'):
        f.seek(0)
        return gzip.GzipFile(fileobj=f)
    else:
        f.seek(0)
        return f
f = opener(raw_src)

In [ ]:
ps = f.read()
f.close()

In [ ]:
with opener(raw_src) as f:
    ps = f.read().decode('iso-8859-1')

In [ ]:
len(ps)

In [ ]:
list(d_metadata.items())[0]

In [ ]:
params_

In [ ]:
n_results

In [ ]:
len(d_metadata.keys())

In [ ]:
next(a)

In [ ]:
print()
print(80 * '*')
print(xml_text)


In [ ]:

for link in article_metadata.find_all('link'):
    pass

In [ ]:

if 'author' in link.attrs:
    k, v = link.attrs['title'], link.attrs['href']
    d_links[k] = v

In [ ]:
link.attrs

In [ ]:
v

In [ ]:
link

In [ ]:
link.attrs

In [ ]:
article_metadata.find_all('link')[0].attrs

In [ ]:

summary = ''.join(soup.find('summary').contents).strip().replace('\n', ' ')
title = ''.join(soup.find('title').contents).strip().replace('\n', ' ')
title

In [ ]:

soup.find_all('title')

In [ ]:
'http://export.arxiv.org/api/query?search_query=all:electron&start=0&max_results=10'

In [ ]:
import xml

In [ ]:
e = xml.etree.ElementTree.parse('xml.xml')

In [ ]:
e.findall('title')

In [ ]:
bf.data(r.text)

In [ ]:
r.text

In [ ]:
import urllib

base = 'http://export.arxiv.org/api/'
method = 'query'
search_term = 'electron'
parameters
parameters = 'search_query=all:' + electron&start=0&max_results=10'
query = base + method + '?' + parameters
data = urllib.request.urlopen(query).read()
print(data)

In [ ]:
data.